In [1]:
!wget https://raw.githubusercontent.com/savasy/TurkishTextClassification/master/TTC4900.csv


--2025-06-09 13:15:31--  https://raw.githubusercontent.com/savasy/TurkishTextClassification/master/TTC4900.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10627541 (10M) [text/plain]
Saving to: ‘TTC4900.csv.1’

TTC4900.csv.1       100%[===================>]  10.13M  --.-KB/s    in 0.1s    

2025-06-09 13:15:31 (103 MB/s) - ‘TTC4900.csv.1’ saved [10627541/10627541]



In [2]:
import pandas as pd
data= pd.read_csv("TTC4900.csv")
data=data.sample(frac=1.0, random_state=42)
data

,category,text
4657,teknoloji,acıların kedisi sam çatık kaşlı kedi sam in i...
3539,spor,g saray a git santos van_persie den forma ala...
907,dunya,endonezya da çatışmalar 14 ölü endonezya da i...
4353,teknoloji,emniyetten polis logolu virüs uyarısı telefon...
3745,spor,beni türk yapın cristian_baroni yıldırım dan ...
...,...,...
4426,teknoloji,kuantum bilgisayarlar için dev adım amerikalı...
466,siyaset,koç tan kürt sorunu ile ilgili önemli açıklam...
3092,saglik,her yıl 125 bin bebek prematüre doğuyor türki...
3772,spor,g saray da borç alacak farkı büyük ! galatasa...


In [3]:
labels=["teknoloji","ekonomi","saglik","siyaset","kultur","spor","dunya"]
id2label={i:l for i,l in enumerate(labels)}
label2id={l:i for i,l in enumerate(labels)}
data["labels"]=data.category.map(lambda x: label2id[x.strip()])
data.head()

,category,text,labels
4657,teknoloji,acıların kedisi sam çatık kaşlı kedi sam in i...,0
3539,spor,g saray a git santos van_persie den forma ala...,5
907,dunya,endonezya da çatışmalar 14 ölü endonezya da i...,6
4353,teknoloji,emniyetten polis logolu virüs uyarısı telefon...,0
3745,spor,beni türk yapın cristian_baroni yıldırım dan ...,5


In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
NUM_LABELS = len(labels)

In [6]:
from transformers import BertTokenizerFast, BertForSequenceClassification

tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-turkish-uncased",
                                              max_length=512)

model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased",
                                                      num_labels=NUM_LABELS,
                                                      id2label=id2label,
                                                      label2id = label2id)
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [7]:
SIZE= data.shape[0]
(3*SIZE )//4

3675

In [8]:
SIZE= data.shape[0]  # 4900 samples

## sentences
train_texts= list(data.text[:SIZE//2]) # [: 2450]
val_texts= list(data.text[SIZE//2:(3*SIZE)//4 ]) # [2450:3675 ]
test_texts= list(data.text[(3*SIZE)//4:]) # [3675 : ]

## labels
train_labels= list(data.labels[:SIZE//2])  # [: 2450]
val_labels= list(data.labels[SIZE//2:(3*SIZE)//4])   # [2450:3675 ]
test_labels= list(data.labels[(3*SIZE)//4:])  # [3675 : ]

## check the size
len(train_texts), len(val_texts), len(test_texts)

(2450, 1225, 1225)

In [9]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [10]:
import torch
from torch.utils.data import Dataset
class MyDataset(Dataset):
   def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

   def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

   def __len__(self):
        return len(self.labels)

In [11]:
train_dataset = MyDataset(train_encodings, train_labels)
val_dataset = MyDataset(val_encodings, val_labels)
test_dataset = MyDataset(test_encodings, test_labels)

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./TTC4900Model',
    do_train=True,
    do_eval=True,
    report_to='none',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./multi-class-logs',
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    fp16=True,
    load_best_model_at_end=True
)

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  Precision, Recall, f1, _ =   precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': Precision,
        'Recall': Recall
    }

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics= compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
q=[trainer.evaluate(eval_dataset=data) for data in [train_dataset, val_dataset, test_dataset]]

pd.DataFrame(q, index=["train","val","test"]).iloc[:,:5]

In [ ]:
pd.DataFrame(q, index=["train","val","test"])

In [ ]:
def predict(text):
    inputs = tokenizer(text, padding=True,truncation=True, max_length=512, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs, probs.argmax(), model.config.id2label[probs.argmax().item()]

In [ ]:
text = "Fenerbahçeli futbolcular kısa paslarla hazırlık çalışması yaptılar" # englis translate : Fenerbahce football players made preparations with short passes
predict(text) # sports in english

In [ ]:
model_path = "turkish-text-classification-model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast

model_path = "turkish-text-classification-model"
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
turkish_classifier= pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
turkish_classifier("Sinemada hangi filmler oynuyor bugün")  #  What movies are playing in the cinema today?
# kultur == culture

In [ ]:
turkish_classifier("Dolar ve Euro bugün yurtiçi piyasalarda yükseldi") # Dollar and Euro in domestic markets today rose


In [ ]:
turkish_classifier("Bayern Münih ile Barcelona bugün karşı karşıya geliyor. Maçı İngiliz hakem James Watts yönetecek!") # Bayern Munich vs Barcelona today
